<a href="https://colab.research.google.com/github/giustinod/Fine-Tuning-Llama-2LLM/blob/main/Fine_tune_Llama_3_2_3B_Instruct_with_Unsloth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Genera il dataset a partire dai file json nella directory ./data

In [ ]:
!pip install pytesseract
!pip install pdf2image
!apt-get install poppler-utils

import pytesseract
from pdf2image import convert_from_path
import glob

drive_path = "/content/drive/MyDrive/Colab Notebooks/data"

pdfs = glob.glob(drive_path + "/ocr/*.pdf")

for pdf_path in pdfs:
    print(pdf_path)
    pages = convert_from_path(pdf_path, 500)

    for pageNum,imgBlob in enumerate(pages):
        text = pytesseract.image_to_string(imgBlob,lang='eng')

        with open(f'{pdf_path}.txt', 'a') as the_file:
            the_file.write(text)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.7 [186 kB]
Fetched 186 kB in 2s (118 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126101 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.7_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.7) ...
Setting up poppler-utils (22.02.0-2ubuntu0.7) ...
Processing triggers for man-db (2.10.2-1) ...
/content/drive/MyDrive/Colab Notebooks/data/ocr/3.3 Disp. 15-2013 Allegato 1.pdf


In [6]:
!pip install datasets

from datasets import Dataset
from string import Template
import pandas as pd
import json
import os
import wandb
from google.colab import userdata

hf_token = userdata.get('HF_TOKEN')

drive_path = "/content/drive/MyDrive/Colab Notebooks/data"

df = pd.DataFrame(columns=['instruction', 'input', 'output'])

instruction_spec = Template('Write test expressions for project $project, group: $group_id and object: $object')
input_spec = Template('Requirements:\n$requirements')
output_spec = Template('Test expressions:\n$tests')

instruction_var = Template('Describe variable: $variable for project $project')
output_var = Template('Object type: $obj_type\nDescription: $var_description\nType: $var_type\nValue range: $range\nValue description: $val_description')

def buildDataframeForSpec(json_file, project, df):
  with open(drive_path + '/' + json_file) as jf:
    data = json.load(jf)
    for elem in data:
      p = instruction_spec.substitute(project = project, group_id = elem['group_id'], object = elem['object'])
      # build requirements string
      reqString = ''
      reqNo = 0
      req_list = elem['requirements']
      for req in req_list:
        if reqNo > 0:
          reqString = reqString + '\n'
        if req['id']:
          reqString = reqString + req['id'] + ' - ' + req['description']
        else:
          reqString = reqString + req['description']
        reqNo = reqNo + 1
      # build tests string
      testNo = 0
      testString = ''
      test_list = elem['tests']
      for test in test_list:
        if testNo > 0:
          testString = testString + '\n'
        testString = testString + test
        testNo = testNo + 1
      i = input_spec.substitute(requirements = reqString, tests = testString)
      o = output_spec.substitute(requirements = reqString, tests = testString)
      df = pd.concat([df, pd.DataFrame([{'instruction': p, 'input': i, 'output': o }])], ignore_index=True)
    return df

def buildDataframeForVariables(json_file, project, df):
  with open(drive_path + '/' + json_file) as jf:
    data = json.load(jf)
    for elem in data:
      if elem['variable_name'] and elem['variable_name'] != '':
        # build premises
        p = instruction_var.substitute(variable = elem['variable_name'], project = project)
        # build conclusion
        valDescr = '\n'
        val_descriptions = elem['value_descriptions']
        for key, value in val_descriptions.items():
          valDescr = valDescr + str(key) + ': ' + value + '\n'
        o = output_var.substitute(obj_type = elem['object_type'], var_description = elem['variable_description'], var_type = elem['type'], range = elem['value_range'], val_description = valDescr)
        df = pd.concat([df, pd.DataFrame([{'instruction': p, 'input': '', 'output': o }])], ignore_index=True)
    return df

if __name__ == "__main__":
  for jf in os.listdir(drive_path):
    if jf.endswith('spec.json'):
      df = buildDataframeForSpec(jf, jf.split('_')[0], df)
      print(jf + ' ok!')
    elif jf.endswith('variables.json'):
      df = buildDataframeForVariables(jf, jf.split('_')[0], df)
      print(jf + ' ok!')
    else:
      print(jf + ' skipped!')

  # Push dataset on HuggingFace
  dataset = Dataset.from_pandas(df)
  dataset.push_to_hub("azservice/specs_and_variables_2", token = hf_token)
  print('Pushed on HF')

.ipynb_checkpoints skipped!
ILL2_CF650_RSA_Variables.pdf skipped!
RouteProtection.pdf skipped!
RouteLocking.pdf skipped!
partialSSRS.pdf skipped!
project1_spec.json ok!
project2_variables.json ok!
project1_variables.json ok!
project2_spec.json ok!
3.3 Disp. 15-2013 Allegato 1.pdf skipped!
SCDS_Route_Protection_RSA.docx skipped!


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/331 [00:00<?, ?B/s]

Pushed on HF


check also: https://medium.com/@amrstech/fine-tuning-made-easy-with-unsloth-and-colab-e0993f3f4c07

check also: https://wandb.ai/capecape/alpaca_ft/reports/How-to-Fine-Tune-an-LLM-Part-1-Preparing-a-Dataset-for-Instruction-Tuning--Vmlldzo1NTcxNzE2

In [ ]:
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

from huggingface_hub import login
from google.colab import userdata
import wandb

import torch
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM
from datasets import load_dataset
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported

hf_token = userdata.get('HF_TOKEN')
login(hf_token)

wb_token = userdata.get("wandb")

wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune Llama-3.2-3B-Instruct on AZService', #
    job_type="training",
    anonymous="allow"
)

# caricamento modello e test
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    # deepseek-ai/deepseek-math-7b-instruct, mistralai/Mathstral-7B-v0.1, unsloth/Llama-3.2-3B-Instruct
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    # max_seq_length = max_seq_length,
    # dtype = dtype,
    # load_in_4bit = load_in_4bit,
    token = hf_token,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

alpaca_prompt = """Write as response the test expressions corresponding to given project, object and requirements given in input.

### Input:
{}

### Response:
{}"""

def formatting_prompts_func(examples):
    project      = examples["project"]
    inputs       = examples["premises"]
    outputs      = examples["conclusion"]
    texts = []
    for project, input, output in zip(project, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(project, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

def prompt_no_input(instruction, output):
  return ("Below is an instruction that describes a task. "
          "Write a response that appropriately completes the request.\n\n"
          "### Instruction:\n{}\n\n### Response:\n{}").format(instruction, output)

def prompt_input(instruction, input, output):
  return ("Below is an instruction that describes a task, paired with an input that provides further context. "
          "Write a response that appropriately completes the request.\n\n"
          "### Instruction:\n{}\n\n### Input:\n{}\n\n### Response:\n{}").format(instruction, input, output)

def create_prompt(rows):
    instructions = rows["instruction"]
    inputs       = rows["input"]
    outputs      = rows["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt_no_input(instruction, output) if input == "" else prompt_input(instruction, input, output)
        texts.append(text + EOS_TOKEN)
    return { "text" : texts, }
pass

dataset = load_dataset("azservice/specs_and_variables_2", split="train", token = hf_token)
dataset = dataset.map(create_prompt, batched = True,)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    # dataset_text_field = "text",
    # max_seq_length = max_seq_length,
    # dataset_num_proc = 2,
    # packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        # neftune_noise_alpha = 5,
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        # warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps = 1,
        # learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        # logging_steps = 1,
        # optim = "adamw_8bit",
        # weight_decay = 0.01,
        # lr_scheduler_type = "linear",
        # seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

import os
os.environ['UNSLOTH_RETURN_LOGITS'] = '1'

trainer.train()

model = FastLanguageModel.for_inference(model)

messages = [
    {"role": "user", "content": "Write test expressions for project1, object GRB and following requirement: If a critical command is activated, the critical confirmation of that command must be acknowledged by the ATS operator within the designated time."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids=inputs, streamer=text_streamer, max_new_tokens=128, use_cache=True)

# Save locally as sharded model files
model.save_pretrained_merged("TestLogica", tokenizer, save_method = "merged_16bit",)
# Push to Huggingface hub - replace the space username as required
model.push_to_hub_merged("azservice/TestLogica-Llama-3.2-3B-Instruct", tokenizer, save_method = "merged_16bit", token = hf_token)


Found existing installation: unsloth 2025.3.19
Uninstalling unsloth-2025.3.19:
  Successfully uninstalled unsloth-2025.3.19
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-n0_tu913
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-n0_tu913
  Resolved https://github.com/unslothai/unsloth.git to commit 6c234d5a66adb76b9b93fb0f2445648199d88e66
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.3.19-py3-none-any.whl size=192661 sha256=aadc41151822417b6392b1b4f767475c97ff86b1d685c72aee929e2e359c7530
  Stored in directory: /tmp/pip-ephem-wheel-cache-4wgzgy3j/wheels/d1/17/05/850ab10c33284a4763b0595cd8ea9d01fce6e221cac24b3c01
Successfully built unsloth


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/9734 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 9,734 | Num Epochs = 3 | Total steps = 3,648
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)


...

Unsloth: Saving to organization with address azservice/TestLogica-Llama-3.2-3B-Instruct
Unsloth: Saving tokenizer... Done.
Unsloth: Saving to organization with address azservice/TestLogica-Llama-3.2-3B-Instruct
Unsloth: Saving azservice/TestLogica-Llama-3.2-3B-Instruct/pytorch_model-00001-of-00002.bin...
Unsloth: Saving azservice/TestLogica-Llama-3.2-3B-Instruct/pytorch_model-00002-of-00002.bin...
Unsloth: Uploading all files... Please wait...


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/azservice/TestLogica-Llama-3.2-3B-Instruct


Il tentavivo è di usare un modello specifico per operazioni metematiche

cfr. https://medium.com/@vinyes.marina/large-language-models-and-math-a-review-of-approaches-and-progress-b58c76e7716e

cfr. https://medium.com/@marvashahid09/fine-tuning-deepseekmath-for-solving-math-problems-10997bc095d4

In [ ]:
%pip install gradio

import gradio as gr
from huggingface_hub import InferenceClient

# Safely initialize the inference client
def initialize_client():
    try:
        print("Initializing inference client...")
        client = InferenceClient("mistralai/Mathstral-7B-v0.1")
        print("Inference client initialized successfully!")
        return client
    except Exception as e:
        print(f"Error initializing inference client: {e}")
        return None

client = initialize_client()

# Chatbot response logic
def respond(
    message,
    history: list[tuple[str, str]],
    system_message,
    max_tokens,
    temperature,
    top_p,
):
    if not client:
        return "Error: Inference client not initialized."

    messages = [{"role": "system", "content": system_message}]

    # Add historical interactions
    for val in history:
        if val[0]:
            messages.append({"role": "user", "content": val[0]})
        if val[1]:
            messages.append({"role": "assistant", "content": val[1]})

    # Add user message
    messages.append({"role": "user", "content": message})

    try:
        print("Sending request to model...")
        response = client.chat_completion(
            messages,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
        ).choices[0].message.content
        print("Response received successfully!")
        return response
    except Exception as e:
        print(f"Error during inference: {e}")
        return f"Error during inference: {e}"

# Gradio interface
def launch_demo():
    try:
        demo = gr.ChatInterface(
            respond,
            additional_inputs=[
                gr.Textbox(value="You are a friendly Chatbot. Your name is Juninho.", label="System message"),
                gr.Slider(minimum=1, maximum=2048, value=512, step=1, label="Max new tokens"),
                gr.Slider(minimum=0.1, maximum=4.0, value=0.7, step=0.1, label="Temperature"),
                gr.Slider(
                    minimum=0.1,
                    maximum=1.0,
                    value=0.95,
                    step=0.05,
                    label="Top-p (nucleus sampling)",
                ),
            ],
        )
        demo.launch()
    except Exception as e:
        print(f"Error launching Gradio app: {e}")

if __name__ == "__main__":
    launch_demo()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.1 MB/s eta 0:00:00
Initializing inference client...
Inference client initialized successfully!


/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:338: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5fbcf95f644cef2bef.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
